In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import csv
import re

chrome_options = webdriver.ChromeOptions() 
chrome_options.add_argument("--blink-settings=imagesEnabled=false") # disabling images to improve run time 
chrome_options.add_argument('--headless')  # Enable headless mode
chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration in headless mode (optional)
driver = webdriver.Chrome(options=chrome_options)



fields = ["Link", "Title", "Year", "Rating"] # creating data columns
filename = "Letterboxd_scrape_first500.csv" # new file
# creating lists 
rows = []
links = []
titles_temp = [] #temporary container for movie info 
titles = []
year = []
rating = []
errors = []

for i in range(1,500):
    url = f"https://letterboxd.com/films/popular/page/{i}/"
    driver.get(url)
    try:
        #movies = driver.find_elements(By.XPATH, "//*[@id='films-browser-list-container']/ul/li") # container of all movies on page
        # i put by css selector because it is supposed to be faster
        movies = driver.find_elements(By.CSS_SELECTOR, "#films-browser-list-container ul li")
        for movie in movies: # iterate through movies 
            movie_info = movie.find_element(By.TAG_NAME, "a").get_attribute('data-original-title')
            if movie_info.find("(") == -1 or movie_info[-1].isdigit() == False:
                continue
            titles_temp.append(movie_info) # movie info
            links.append(movie.find_element(By.TAG_NAME, "a").get_attribute('href')) # link
    except: #print page number of where error occurs
        errors.append(i)
        continue
for i in errors:
    url = f"https://letterboxd.com/films/popular/page/{i}/"
    driver.get(url)
    try:
        #movies = driver.find_elements(By.XPATH, "//*[@id='films-browser-list-container']/ul/li") # container of all movies on page
        # i put by css selector because it is supposed to be faster
        movies = driver.find_elements(By.CSS_SELECTOR, "#films-browser-list-container ul li")
        for movie in movies: # iterate through movies 
            
            movie_info = movie.find_element(By.TAG_NAME, "a").get_attribute('data-original-title')
            if movie_info.find("(") == -1 or movie_info[-1].isdigit() == False:
                continue
            titles_temp.append(movie_info) # movie info
            links.append(movie.find_element(By.TAG_NAME, "a").get_attribute('href')) # link
    except: #print page number of where error occurs
        print("page: "+str(i))
        continue
for film in titles_temp: #iterate through movie info for each one
    try:
        new_list = re.split(r'[\(\)]',film) # split string at parentheses into a new list 
        new_list = [item.strip() for item in new_list] # strip all parentheses of spaces
# append separate features of the movie info 
        titles.append(new_list[0]) 
        year.append(new_list[1])
        rating.append(new_list[2])
    # print movie info when there's an error to track it 
    except IndexError:
        print(new_list)
        print(film)
        continue
for i in range(len(titles)-1): 
    try:
        rows.append([links[i], titles[i], year[i], rating[i]]) # create rows of csv 
    except IndexError:
        print(i) # print index where error occurs 
with open(filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields) 
    csvwriter.writerows(rows)

driver.quit()
print("Done")
# done message is because you dont see the webscraping in action in headless mode
# 500 pages should take rougly 20ish minutes 

Done
